## Prepare Premier Database Schema

This notebook reformats the premier database schema to be compatible with the Text2SQL pipeline.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jupyter_black

jupyter_black.load()

In [13]:
# import pandas as pd
# import os
import datetime
import re

DB_SCHEMA_NAME = "rwdex_raw_premier_hospital"
INPUT_SCHEMA_PATH = "../databases/premier/original/sample_dummy_data_premier.txt"
OUTPUT_SCHEMA_PATH = "../databases/premier/db_tables_info.txt"

CREATE_TABLE_TEMPLATE = """
CREATE TABLE {schema_name}.{table_name} (
{columns_info}
)

/*
3 rows from {table_name} table:
{samples_info}
*/
"""

In [8]:
def replace_with_date(match):
    """Function to replace each match with the corresponding date string"""
    year, month, day = match.groups()
    return f"{year}-{month}-{day}"


def get_new_table_info(table_info, create_table_template):
    """Generates new format for the table info."""

    # Get table name from first line
    tbl_name, other_info = table_info.split("\n", 1)
    tbl_name = tbl_name.replace("Table Name: ", "").strip()

    # Get columns and sample rows
    cols_info, rows_info = other_info.replace("Columns:\n", "").split("Sample Rows:\n")

    cols_info = cols_info.strip().split("\n")
    col_names = [col.split(": ")[0] for col in cols_info]
    col_dtypes = [col.split(": ")[1] for col in cols_info]
    col_dtypes = [col.upper() for col in col_dtypes]

    col_dtypes = [
        col.replace("CHARACTER VARYING", "VARCHAR(256)") for col in col_dtypes
    ]
    col_dtypes = [
        col.replace("TIMESTAMP WITHOUT TIME ZONE", "DATE") for col in col_dtypes
    ]

    columns_info = [
        f"\t{col_name} {col_dtypes[i]}" for i, col_name in enumerate(col_names)
    ]
    columns_info = ", \n".join(columns_info)

    rows_info = rows_info.split("\n")[:3]

    # Regular expression pattern to match datetime.datetime expressions
    pattern = r"datetime\.datetime\((\d+), (\d+), (\d+), \d+, \d+\)"
    rows_info2 = []
    for row in rows_info:
        # Replace all matches in the input string
        formatted_row = re.sub(pattern, replace_with_date, row)
        rows_info2.append(formatted_row)
    rows_info = rows_info2

    rows_info = [row.strip(")") for row in rows_info]
    rows_info = [row.strip("(") for row in rows_info]
    rows_info = [row.replace(", ", "\t") for row in rows_info]

    rows_info = "\n".join(rows_info)
    rows_info = "\t".join(col_names) + "\n" + rows_info

    tbl_info = create_table_template.replace("{schema_name}", DB_SCHEMA_NAME)
    tbl_info = tbl_info.replace("{table_name}", tbl_name)
    tbl_info = tbl_info.replace("{columns_info}", columns_info)
    tbl_info = tbl_info.replace("{samples_info}", rows_info)

    #     print(tbl_info)
    #     print("-" * 50)

    #     print(f"Table Name: {tbl_name}")
    #     print("-" * 50)
    #     print(f"Columns Info: \n{cols_info}")
    #     print("-" * 50)
    #     print(f"Column Names: \n{col_names}")
    #     print("-" * 50)
    #     print(f"Column Dtypes: \n{col_dtypes}")
    #     print("-" * 50)
    #     print(f"Rows Info: \n{rows_info}")
    #     print("*" * 50)

    return tbl_info

In [15]:
with open(INPUT_SCHEMA_PATH, "r") as fp:
    schema = fp.readlines()
    schema = "".join(schema).strip()
    schema = schema.split("\n\n")
    schema = [tbl_info.strip() for tbl_info in schema]

tables_info = []
for table in schema:
    table_info = get_new_table_info(table, create_table_template=CREATE_TABLE_TEMPLATE)
    tables_info.append(table_info)

tables_info = "\n\n".join(tables_info)
print(tables_info[:1000])


CREATE TABLE rwdex_raw_premier_hospital.admtype (
	adm_type SMALLINT, 
	adm_type_desc VARCHAR(256)
)

/*
3 rows from admtype table:
adm_type	adm_type_desc
1	'EMERGENCY'
2	'URGENT'
3	'ELECTIVE'
*/



CREATE TABLE rwdex_raw_premier_hospital.aprdrg (
	apr_drg SMALLINT, 
	apr_drg_desc VARCHAR(256)
)

/*
3 rows from aprdrg table:
apr_drg	apr_drg_desc
711	'POST-OPERATIVE	POST-TRAUMA	OTHER DEVIC'
720	'SEPTICEMIA AND DISSEMINATED INFECTIONS'
721	'POST-OPERATIVE	POST-TRAUMATIC	OTHER DE'
*/



CREATE TABLE rwdex_raw_premier_hospital.chgmstr (
	std_dept_code SMALLINT, 
	prod_name_meth_desc VARCHAR(256), 
	prod_name_meth_code VARCHAR(256), 
	prod_name_desc VARCHAR(256), 
	prod_name_code VARCHAR(256), 
	prod_class_desc VARCHAR(256), 
	prod_class_code VARCHAR(256), 
	prod_cat_desc VARCHAR(256), 
	prod_cat_code VARCHAR(256), 
	sum_dept_desc VARCHAR(256), 
	std_dept_desc VARCHAR(256), 
	clin_sum_desc VARCHAR(256), 
	clin_sum_code VARCHAR(256), 
	clin_dtl_desc VARCHAR(256), 
	std_chg_desc VARCHAR(256)

In [16]:
# Write to txt file
with open(OUTPUT_SCHEMA_PATH, "w") as fp:
    fp.write(tables_info)